# Series Temporales

Autor: Carlos Sevilla Barceló

__Indice__

1. Un poco de teoría
2. Estacionario
    1. Test
    2. Transformaciones
3. ACF y PACF
4. Modelos clásicos
    1. AR
    2. MA
    3. ARMA
    4. ARIMA
    5. Otros
5. Modelos ML
    1. Introducción
    1. Transformacion
    2. Ejemplos





In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import statsmodels.api as sm

# Teoría 

Antes de empezar a trabajar con modelos de series de tiempo es importante entender algunos conceptos clave:

Una serie temporal es una colección de observaciones de una variable o conjunto de variables, ordenadas en el tiempo y espaciadas uniformemente. Las series temporales son utilizadas en muchos campos, incluyendo la economía, la ingeniería, las ciencias sociales y la meteorología, para analizar y predecir patrones y tendencias en los datos.

Una serie temporal tiene varias propiedades importantes que deben tenerse en cuenta al analizarla:

* __Tendencia__: una serie temporal puede tener una tendencia a largo plazo, que es una dirección general en la que los datos se mueven. Puede ser ascendente o descendente y puede ser lineal o no lineal.

* __Estacionalidad__: las series temporales pueden tener patrones estacionales o cíclicos, lo que significa que los datos varían según el momento del año o del ciclo de negocio. La estacionalidad puede ser aditiva (los datos aumentan o disminuyen en la misma cantidad en cada ciclo) o multiplicativa (los datos aumentan o disminuyen en un factor constante en cada ciclo).

* __Ciclos__: las series temporales pueden tener ciclos más largos que la estacionalidad, que pueden durar varios años. Por ejemplo, las recesiones económicas y las expansiones pueden ser ciclos que se repiten en una serie temporal.

* __Ruido__: las series temporales a menudo tienen una componente de ruido o variación aleatoria, que se debe a factores impredecibles que no se pueden modelar fácilmente. El ruido puede dificultar la identificación de patrones y tendencias en los datos.

* __Autocorrelación__: la autocorrelación se refiere a la relación entre los valores de una variable en diferentes momentos en el tiempo. Si una serie temporal tiene una alta autocorrelación, significa que los valores están fuertemente relacionados con los valores anteriores y posteriores. La autocorrelación puede ser útil para identificar patrones en la serie temporal.

* __Estacionariedad__: una serie temporal se considera estacionaria si sus propiedades estadísticas no cambian con el tiempo. Esto significa que la media, la varianza y la autocorrelación son constantes a lo largo del tiempo. La estacionariedad es importante para aplicar ciertos modelos estadísticos y para hacer predicciones precisas en la serie temporal.

In [ ]:
sm.datasets.sunspots.load_pandas().data

In [ ]:
sunspots = sm.datasets.sunspots.load_pandas().data.dropna()

fig, ax = plt.subplots(figsize=(16, 7))
ax.plot(sunspots['YEAR'],sunspots['SUNACTIVITY'])
ax.set_xlabel('Year')
ax.set_ylabel('Sun activity')
fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
co2 = sm.datasets.co2.load_pandas().data.dropna()

fig, ax = plt.subplots(figsize=(16, 7))
ax.plot(co2['co2'])
ax.set_xlabel('Time')
ax.set_ylabel('CO2 concentration (ppmw)')
fig.autofmt_xdate()
plt.tight_layout()

# Estacionariedad

La estacionariedad es una propiedad importante en el análisis de series temporales. Una serie temporal se considera estacionaria si sus propiedades estadísticas no cambian con el tiempo. Esto significa que la media, la varianza y la autocorrelación son constantes a lo largo del tiempo.

En otras palabras, si una serie temporal es estacionaria, entonces su patrón general, incluyendo su nivel medio y su variabilidad, permanece constante a lo largo del tiempo. Esto facilita el modelado y la predicción de la serie temporal, ya que se pueden utilizar técnicas estadísticas simples y los resultados obtenidos en una parte de la serie temporal pueden ser extrapolados a otras partes.

![alt text](img/stationary.webp "Title")

Es importante destacar que la estacionariedad es una propiedad matemática y no depende de la interpretación del problema o de la naturaleza de los datos. Es posible que una serie temporal tenga una tendencia o un patrón estacional y aún así ser estacionaria en términos estadísticos.

Por lo tanto, al analizar una serie temporal, es importante determinar si es estacionaria o no, ya que esto puede tener implicaciones importantes en la elección de los métodos de análisis y en la precisión de las predicciones que se pueden hacer.

Algunos modelos estadísticos exigen que la serie a modelar cumpla la condición de estacionariedad, con lo que antes de trabajar con estos modelos de series de tiempo es importante asegurarse de que la serie de tiempo sea estacionaria. En esta sección exploraremos dos formas de comprobar si una serie de tiempo es estacionaria y cómo transformar la serie de tiempo en caso de que no lo sea.

## Tests

Para comprobar si una serie de tiempo es estacionaria podemos utilizar el test de Dickey-Fuller aumentado (ADF) y Kwiatkowski-Phillips-Schmidt-Shin (KPSS)

### ADF 
Este test comprueba si la serie de tiempo tiene raíces unitarias, lo que implica que la serie de tiempo no es estacionaria.

La intuición que subyace a una prueba de raíz unitaria es que determina hasta qué punto una serie temporal está definida por una tendencia.

Existen varias pruebas de raíz unitaria y la de Dickey-Fuller aumentada puede ser una de las más utilizadas. Utiliza un modelo autorregresivo y optimiza un criterio de información a través de múltiples valores de retardo diferentes.

La hipótesis nula de la prueba es que la serie temporal puede estar representada por una raíz unitaria, que no es estacionaria (tiene alguna estructura dependiente del tiempo). La hipótesis alternativa (que rechaza la hipótesis nula) es que la serie temporal es estacionaria.

* Hipótesis nula (H0): Si no se rechaza, sugiere que la serie temporal tiene una raíz unitaria, lo que significa que __no es estacionaria__. Tiene una estructura dependiente del tiempo.
* Hipótesis alternativa (H1): Si se rechaza la hipótesis nula, sugiere que la serie temporal no tiene una raíz unitaria, lo que significa que __si es estacionaria__. No tiene estructura dependiente del tiempo.

Interpretamos este resultado utilizando el valor p de la prueba. Un valor p inferior a un umbral (como el 5% o el 1%) indica que se rechaza la hipótesis nula (estacionariedad); de lo contrario, un valor p superior al umbral indica que no se rechaza la hipótesis nula (no estacionariedad).

* Valor p > 0,05: No se rechaza la hipótesis nula (H0), los datos tienen una raíz unitaria y no son estacionarios.
* Valor p <= 0,05: Rechazamos la hipótesis nula (H0), los datos no tienen una raíz unitaria y son estacionarios.

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Test ADF para sunspot
result = adfuller(sunspots['SUNACTIVITY'])

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))


In [ ]:
# Test ADF para CO2
result = adfuller(co2['co2'])

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
 print('\t%s: %.3f' % (key, value))

### KPSS

El KPSS es otra prueba para comprobar la estacionariedad de una serie temporal. Las hipótesis nula y alternativa de la prueba KPSS son opuestas a las de la prueba ADF.

* Hipótesis nula (H0): El ciclo es estacionario en cuanto a la tendencia.
* Hipótesis alternativa (H1): La serie tiene una raíz unitaria (la serie no es estacionaria).

In [ ]:
from statsmodels.tsa.stattools import kpss

In [ ]:
result = kpss(sunspots['SUNACTIVITY'])

print('KPSS Test: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[3].items():
 print('\t%s: %.3f' % (key, value))

In [ ]:
result

In [ ]:
result = kpss(co2)

print('KPSS Test: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[3].items():
 print('\t%s: %.3f' % (key, value))

### Que hacer cuando los test difieren


Siempre es mejor aplicar ambas pruebas para asegurarse de que la serie es realmente estacionaria. Los posibles resultados de la aplicación de estas pruebas de estacionariedad son los siguientes:

* Caso 1: Ambas pruebas concluyen que la serie no es estacionaria - La serie no es estacionaria
* Caso 2: Ambas pruebas concluyen que la serie es estacionaria - La serie es estacionaria
* Caso 3: El KPSS indica estacionariedad y el ADF indica no estacionariedad - La serie es estacionaria por tendencia. Es necesario eliminar la tendencia para que la serie sea estacionaria estricta. Se comprueba la estacionariedad de la serie sin tendencia.
* Caso 4: KPSS indica no estacionariedad y ADF indica estacionariedad - La serie es estacionaria por diferencia. Se utilizará la diferenciación para hacer estacionaria la serie. Se comprueba la estacionariedad de la serie diferenciada.



## Transformaciones

Si una serie de tiempo no es estacionaria, podemos aplicar transformaciones para hacerla estacionaria. Algunas de las transformaciones más comunes son:

* Diferenciación: restar la observación actual de la observación anterior para obtener la diferencia. Si es necesario, se pueden aplicar múltiples diferenciaciones hasta que la serie de tiempo sea estacionaria.
* Eliminación de la tendencia.
* Transformación logarítmica: tomar el logaritmo de la serie de tiempo para suavizar las fluctuaciones.
* Transformación de Box-Cox: una transformación que puede ser utilizada para estabilizar la varianza de la serie de tiempo.

### Diferenciación

In [ ]:
co2['co2'].diff()

Que es el equivalente a este código

In [ ]:
co2['co2'] - co2['co2'].shift(1)

### Eliminar la tendencia

Mediante una descomposición STL, puedes obtener el compontente de tendencia de una serie temporal y restarselo. 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Additive Decomposition
result_add = seasonal_decompose(sunspots['SUNACTIVITY'],period=11, model = 'additive')

# Plot
plt.rcParams.update({'figure.figsize': (6,6)})
result_add.plot()

In [ ]:
(sunspots['SUNACTIVITY'] - result_add.trend).plot()

### Transformación Logaritmica

In [ ]:
co2['co2'].plot()

In [ ]:
np.log(co2['co2']).plot()

# ACF y PACF

Los diagramas ACF (Autocorrelation Function) y PACF (Partial Autocorrelation Function) son herramientas comúnmente utilizadas en el análisis de series de tiempo para determinar la presencia de autocorrelación en los datos.

El diagrama ACF muestra la correlación entre una serie de tiempo y sus valores anteriores, es decir, la correlación entre cada valor y los valores precedentes a él. El eje horizontal del diagrama ACF representa el número de retrasos (lags) y el eje vertical representa el valor de correlación. Un pico en el diagrama ACF en el lag k indica una fuerte correlación entre los valores de la serie de tiempo separados por k períodos de tiempo.

Por otro lado, el diagrama PACF muestra la correlación entre una serie de tiempo y sus valores anteriores, teniendo en cuenta solo la influencia directa de los valores en los lags intermedios. En otras palabras, el diagrama PACF muestra la correlación entre dos valores de la serie de tiempo que están separados por k períodos, teniendo en cuenta solo los valores en los lags intermedios. Un pico en el diagrama PACF en el lag k indica que hay una correlación significativa entre los valores de la serie de tiempo separados por k períodos, teniendo en cuenta solo los valores en los lags intermedios.

El uso principal de estos diagramas es determinar la estructura de un modelo de serie de tiempo adecuado para los datos. Al interpretar los picos en los diagramas ACF y PACF, se pueden identificar patrones en los datos que sugieren la presencia de estacionalidad, tendencias, ciclos y otros componentes importantes de la serie de tiempo. Esta información puede ser útil para seleccionar un modelo de serie de tiempo que pueda predecir con precisión los valores futuros de la serie de tiempo.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))
plot_acf(co2['co2'],ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))
plot_pacf(co2['co2'],ax=ax)
plt.show()

## Interpretación

Para obtener el orden de los modelo de tipo ARIMA, seguiremos esta tabla. 

| | AR(p)| MA(q) | ARMA(p,q) |
|---|---|---|---|
| ACF | varios puntos con coef>0 decayendo | 0 excepto los `q` primeros | varios puntos con coef>0 decayendo |
| PCAF | 0 excepto los `p` primeros | varios puntos con coef>0 | varios puntos con coef>0 |

Observando los correlogramas, buscaremos `el corte` de la serie. Esto lo haremos buscando el primer retardo que no se considere significativo (que esté dentro de la región de  no significancia). Lo que se encuentre por encima de estas bandas, se puede considerar significante, y por debajo, a efectos prácticos equivale a 0. 

El valor de `p` o `q` será el valor del último retraso significativo (postivo o negativo)


### Ejemplo: manchas solares. 

In [ ]:
# Diferenciamos la serie para que sea estacionaria
sunspots_diff = sunspots['SUNACTIVITY'].diff().dropna()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))
plot_acf(sunspots_diff,ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))
plot_pacf(sunspots_diff,ax=ax)
plt.show()

Como en en el ACF vemos el corte en el retraso 2, su valor de `q` será = 1, mientras que en el PACF vemos que el corte está en el retraso 9, con lo que su valor de `p` será = 8.

### Ejemplo: co2

In [ ]:
co2_diff = co2['co2'].diff().dropna()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
plot_acf(co2_diff,ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
plot_pacf(co2_diff,ax=ax)
plt.show()

Como en en el ACF vemos el corte en el retraso 9, su valor de `q` será = 8, mientras que en el PACF vemos que el corte está en el retraso 6, con lo que su valor de `p` será = 5.

Para más ejemplos de interpretación, consulta [este artículo](https://enrdados.netlify.app/post/series-temporales-con-arima-ii/).

# Modelos 

## AR - Auto Regresivos

El modelo autorregresivo (AR) es un modelo estadístico utilizado para analizar y predecir valores en una serie de tiempo. En este modelo, la variable dependiente (o la serie de tiempo) se regresa en sí misma con retrasos (también llamados lags) como variables independientes.

Un modelo autorregresivo de orden p (AR(p)) se define como:

y_t = c + φ_1 y_{t-1} + φ_2 y_{t-2} + ... + φ_p y_{t-p} + ε_t

Donde:

- y_t es la variable dependiente en el tiempo t.
- c es una constante.
- φ_1 a φ_p son los coeficientes autorregresivos que miden la relación entre la variable dependiente en el tiempo actual y sus valores pasados.
- ε_t es el término de error aleatorio en el tiempo t.

El término "orden p" hace referencia a la cantidad de lags utilizados en el modelo. El modelo AR(p) es utilizado para series de tiempo que exhiben cierto grado de autocorrelación, es decir, cuando el valor de la serie en un momento dado está correlacionado con sus valores en momentos anteriores.

Para ajustar el modelo AR(p), se utilizan técnicas de estimación de parámetros para determinar los valores de los coeficientes φ_1 a φ_p y la constante c. Luego, el modelo se puede utilizar para hacer pronósticos sobre los valores futuros de la serie de tiempo.

Un modelo AR(p) y un modelo ARIMA(p,0,0) son iguales.

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

model = AutoReg(sunspots_diff, lags=1)
resultados = model.fit()

In [ ]:
print(resultados.summary())

In [ ]:
predicciones = resultados.predict(start=len(sunspots_diff), end=len(sunspots_diff)+10)

In [ ]:
plt.plot(sunspots_diff[-30:])
plt.plot(predicciones)
plt.show()

## MA - Medias móviles

El modelo de media móvil (MA) es otro modelo estadístico utilizado para analizar y predecir valores en una serie de tiempo. A diferencia del modelo AR, el modelo MA no utiliza valores pasados de la variable dependiente como variables independientes. En su lugar, utiliza un término de error aleatorio en el modelo que se deriva de una media móvil de los errores de pronóstico previos.

Un modelo de media móvil de orden q (MA(q)) se define como:

y_t = c + ε_t + θ_1 ε_{t-1} + θ_2 ε_{t-2} + ... + θ_q ε_{t-q}

Donde:

- y_t es la variable dependiente en el tiempo t.
- c es una constante.
- ε_t es el término de error aleatorio en el tiempo t.
- θ_1 a θ_q son los coeficientes de media móvil que miden la relación entre el término de error aleatorio actual y los errores de pronóstico anteriores.

El término "orden q" hace referencia a la cantidad de errores de pronóstico previos utilizados en el modelo. El modelo MA(q) es utilizado para series de tiempo que exhiben cierto grado de heterocedasticidad (es decir, variación no constante en la varianza), lo que hace que los modelos de regresión lineal ordinarios no sean apropiados.

Para ajustar el modelo MA(q), se utilizan técnicas de estimación de parámetros para determinar los valores de los coeficientes θ_1 a θ_q y la constante c. Luego, el modelo se puede utilizar para hacer pronósticos sobre los valores futuros de la serie de tiempo.

Un modelo MA(q) y un modelo ARIMA(0,0,q) son iguales.

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(sunspots_diff, order=(0,0,8))
resultados = model.fit()

In [ ]:
print(resultados.summary())

In [ ]:
predicciones = resultados.predict(start=len(sunspots_diff), end=len(sunspots_diff)+10)

In [ ]:
plt.plot(sunspots_diff[-30:])
plt.plot(predicciones)
plt.show()

## ARMA - Auto Regresivo con Media movil

El modelo autoregresivo de media móvil (ARMA) es un modelo estadístico que combina las características del modelo autoregresivo (AR) y el modelo de media móvil (MA) para analizar y predecir valores en una serie de tiempo. El modelo ARMA se utiliza para series de tiempo que exhiben tanto la autocorrelación (dependencia entre los valores pasados de la serie de tiempo) como la heterocedasticidad (variación no constante en la varianza).

El modelo ARMA(p, q) se define como:

y_t = c + φ_1 y_{t-1} + φ_2 y_{t-2} + ... + φ_p y_{t-p} + ε_t + θ_1 ε_{t-1} + θ_2 ε_{t-2} + ... + θ_q ε_{t-q}

Donde:

- y_t es la variable dependiente en el tiempo t.
- c es una constante.
- φ_1 a φ_p son los coeficientes autoregresivos que miden la relación entre la variable dependiente actual y los valores pasados de la serie de tiempo.
- ε_t es el término de error aleatorio en el tiempo t.
- θ_1 a θ_q son los coeficientes de media móvil que miden la relación entre el término de error aleatorio actual y los errores de pronóstico anteriores.

El término "orden p" se refiere a la cantidad de valores pasados de la serie de tiempo que se incluyen en el modelo AR, mientras que el término "orden q" se refiere a la cantidad de errores de pronóstico anteriores que se incluyen en el modelo MA.

Para ajustar un modelo ARMA, se utilizan técnicas de estimación de parámetros para determinar los valores de los coeficientes φ_1 a φ_p, θ_1 a θ_q y la constante c. Luego, el modelo se puede utilizar para hacer predicciones sobre los valores futuros de la serie de tiempo.

Un modelo ARMA(p,q) y un modelo ARIMA(p,0,q) son iguales

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(sunspots_diff, order=(1,0,8))
resultados = model.fit()

In [ ]:
print(resultados.summary())

In [ ]:
predicciones = resultados.predict(start=len(sunspots_diff), end=len(sunspots_diff)+10)

In [ ]:
plt.plot(sunspots_diff[-30:])
plt.plot(predicciones)
plt.show()

# Referencias y bibliografía

https://towardsdatascience.com/achieving-stationarity-with-time-series-data-abd59fd8d5a0

https://machinelearningmastery.com/time-series-data-stationary-python/